# Medallion Architecture - Demo

**Cel szkoleniowy:** Zrozumienie koncepcji architektury medalionowej (Bronze/Silver/Gold) i zasad projektowania data lakehouse.

**Zakres tematyczny:**
- Bronze / Silver / Gold - logika warstw
- ETL vs ELT approach
- Zasady projektowania pipeline'ów
- Partitioning strategy
- Audyt i lineage - metadane w każdym kroku
- Data quality w kontekście warstw

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers

## Wstęp teoretyczny - Medallion Architecture

**Cel sekcji:** Dogłębne zrozumienie architektury medalionowej jako fundamentalnego design pattern dla data lakehouse.

---

### Czym jest Medallion Architecture?

**Medallion Architecture** to wielowarstwowy wzorzec organizacji danych w data lakehouse, który dzieli dane na trzy warstwy o rosnącej jakości i wartości biznesowej:

```
┌─────────────────────────────────────────────────────────────┐
│                    DATA SOURCES                              │
│  • Transactional DBs  • APIs  • Files  • IoT  • Streams     │
└───────────────────────┬─────────────────────────────────────┘
                        │ Extract & Load
                        ▼
┌─────────────────────────────────────────────────────────────┐
│           🥉 BRONZE LAYER (Raw / Landing Zone)              │
│                                                              │
│  Charakterystyka:                                            │
│  • Dane "as-is" bez transformacji wartości                  │
│  • Append-only, immutable                                   │
│  • Audit metadata: timestamp, source, user                  │
│  • Multi-format: JSON, CSV, Parquet, Avro                   │
│                                                              │
│  Retention: 3-7 lat (długoterminowa historia)              │
│  SLA: Real-time lub near-real-time                          │
│  Use case: Data recovery, reprocessing, compliance          │
└───────────────────────┬─────────────────────────────────────┘
                        │ Transform (Data Quality)
                        ▼
┌─────────────────────────────────────────────────────────────┐
│       🥈 SILVER LAYER (Cleansed / Validated)                │
│                                                              │
│  Charakterystyka:                                            │
│  • Deduplikacja po kluczu biznesowym                        │
│  • Walidacja: NOT NULL, data types, ranges                  │
│  • Standaryzacja: dates, text, formats                      │
│  • Business rules enforcement                                │
│  • Schema evolution support                                  │
│                                                              │
│  Retention: 1-2 lata (medium-term history)                  │
│  SLA: Hourly lub daily batch                                │
│  Use case: Analytics prep, joins, enrichment                │
└───────────────────────┬─────────────────────────────────────┘
                        │ Transform (Business Logic)
                        ▼
┌─────────────────────────────────────────────────────────────┐
│         🥇 GOLD LAYER (Business / Aggregates)               │
│                                                              │
│  Charakterystyka:                                            │
│  • Pre-aggregated summaries (daily, monthly)                │
│  • Denormalized tables (joins pre-computed)                 │
│  • KPI calculations & business metrics                      │
│  • Star schema / dimensional models                          │
│  • ML feature stores                                         │
│                                                              │
│  Retention: 6-12 miesięcy (short-term, refreshable)        │
│  SLA: Daily lub on-demand                                    │
│  Use case: BI dashboards, reports, ML models                │
└───────────────────────┬─────────────────────────────────────┘
                        │
                        ▼
┌─────────────────────────────────────────────────────────────┐
│                    CONSUMPTION LAYER                         │
│  • Power BI  • Tableau  • Looker  • ML Models               │
└─────────────────────────────────────────────────────────────┘
```

---

### Kluczowe zasady Medallion Architecture

**1. Separation of Concerns**
- Bronze: Data engineers (ingestion, audit)
- Silver: Data engineers + Analytics engineers (quality, standardization)
- Gold: Analytics engineers + Data analysts (business logic, KPIs)

**2. Incremental Processing**
- Każda warstwa procesuje tylko nowe/zmienione dane (watermarking)
- Minimalizacja reprocessingu całych tabel
- MERGE operations dla slowly changing dimensions

**3. Data Quality Gates**
- Bronze → Silver: Schema validation, NOT NULL checks
- Silver → Gold: Business rule validation, referential integrity
- Reject/Quarantine invalid records

**4. Immutability & Auditability**
- Bronze: Immutable landing zone (append-only)
- Audit metadata na każdym etapie (timestamp, source, user)
- Możliwość reprocessingu Silver/Gold z Bronze

**5. Different SLAs per Layer**
- Bronze: Real-time (streaming) lub near-real-time (micro-batch)
- Silver: Hourly lub daily batch
- Gold: Daily lub on-demand refresh

---

### Dlaczego Medallion Architecture?

**✅ Zalety:**

1. **Reprocessing bez re-ingestion**
   - Bronze jako immutable source of truth
   - Silver/Gold można przeliczyć bez ponownego ładowania z source systems

2. **Data Quality Governance**
   - Jasne quality gates między warstwami
   - Tracking rejection rates i invalid records

3. **Performance Optimization**
   - Bronze: Append-only (fast writes)
   - Silver: Partitioned, optimized for joins
   - Gold: Denormalized, optimized for reads (BI queries)

4. **Compliance & Audit**
   - Pełna historia danych w Bronze (GDPR, SOX)
   - Lineage tracking: source → Bronze → Silver → Gold → BI

5. **Cost Optimization**
   - Different retention policies per warstwa
   - Archive/delete older Bronze data po reprocessingu Silver

**⚠️ Trade-offs:**

1. **Storage Cost**: Dane powielone w 3 warstwach
2. **Complexity**: Więcej pipeline'ów do zarządzania
3. **Latency**: Multi-hop processing może zwiększać end-to-end latency

---

### Medallion vs Traditional DWH

| Aspekt | Traditional DWH (ETL) | Medallion (ELT) |
|--------|----------------------|-----------------|
| **Transform Location** | Poza DWH (ETL tools) | W lakehouse (Spark) |
| **Raw Data** | Brak surowych danych | Bronze layer |
| **Schema** | Schema-on-write | Schema-on-read + evolution |
| **Reprocessing** | Re-extract z source | Bronze → Silver/Gold |
| **Cost** | Wysoki (DWH storage) | Niższy (object storage) |
| **Scalability** | Vertical (scale-up) | Horizontal (scale-out) |

---

### Podstawowe pojęcia

- **Medallion Architecture**: Wielowarstwowa architektura danych (Bronze → Silver → Gold)
- **Bronze Layer**: Raw data landing zone - dane bez transformacji, tylko audit metadata
- **Silver Layer**: Cleansed and conformed data - deduplikacja, walidacja, standardization
- **Gold Layer**: Business-level aggregates - modele KPI, reporty, ML features
- **ELT (Extract-Load-Transform)**: Ładuj surowe dane, transformuj w lakehouse
- **Data Lineage**: Śledzenie przepływu danych od źródła do consumption
- **Quality Gates**: Walidacje między warstwami (reject invalid records)

**Dlaczego to ważne?**

Medallion architecture zapewnia:
- **Separation of concerns** - jasne role per warstwa
- **Jasne SLA per warstwa** - różne tempo procesowania
- **Incremental processing** - tylko nowe/zmienione dane
- **Data quality gates** - walidacje na każdym etapie
- **Różne retention policies** - optymalizacja kosztów storage

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta
import uuid

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog jako domyślny
spark.sql(f"USE CATALOG {CATALOG}")

# Ścieżki do danych źródłowych
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"

print(f"\n=== Ścieżki do danych ===")
print(f"Orders: {ORDERS_JSON}")
print(f"Customers: {CUSTOMERS_CSV}")

---

## Sekcja 1: Koncepcja Medallion Architecture

**Wprowadzenie teoretyczne:**

Medallion Architecture dzieli data lakehouse na trzy warstwy o rosnącej jakości danych. Każda warstwa ma określone SLA, retention policy i data quality requirements.

**Kluczowe pojęcia:**
- **Bronze (Raw)**: Append-only, immutable landing zone. Dane "as-is" z systemu źródłowego + audit metadata (ingestion timestamp, source file, version)
- **Silver (Cleansed)**: Validated, deduplicated, standardized. Business rules enforcement, schema evolution, data quality checks
- **Gold (Curated)**: Aggregated, denormalized, business-level. KPI models, reporting tables, ML features, star schema

**Zastosowanie praktyczne:**
- Separacja odpowiedzialności: data engineers (Bronze/Silver), analytics engineers (Gold)
- Incremental processing: tylko nowe/zmienione dane propagowane przez warstwy
- Debug-friendly: możliwość reprocessingu Silver/Gold z Bronze bez re-ingestion

### Przykład 1.1: Bronze Layer - Raw Data Landing

**Cel:** Utworzenie Bronze layer z audit metadata

**Podejście:**
1. Wczytanie surowych danych z JSON
2. Dodanie audit columns: ingest_timestamp, source_file, ingested_by
3. Zapis do Bronze schema bez transformacji biznesowych

In [0]:
# Przykład 1.1 - Bronze Layer (część 1: wczytanie surowych danych)

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

bronze_table = f"{BRONZE_SCHEMA}.orders_bronze"

# Wczytaj surowe dane orders z JSON
orders_raw = (
    spark.read
    .format("json")
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

print("=== Surowe dane (schema) ===")
orders_raw.printSchema()
print(f"\n✓ Wczytano {orders_raw.count()} rekordów z JSON")

# Podgląd surowych danych
display(orders_raw.limit(3))

In [ ]:
# Przykład 1.1 - Bronze Layer (część 2: dodanie audit metadata)

# Dodaj audit metadata (Bronze best practice)
orders_bronze = (
    orders_raw
    .withColumn("_bronze_ingest_timestamp", F.current_timestamp())
    .withColumn("_bronze_source_file", F.input_file_name())
    .withColumn("_bronze_ingested_by", F.lit(raw_user))
    .withColumn("_bronze_version", F.lit(1))
)

print("=== Bronze layer z audit metadata ===")
print("\nAudit columns:")
display(orders_bronze.select(
    "_bronze_ingest_timestamp",
    "_bronze_source_file", 
    "_bronze_ingested_by",
    "_bronze_version"
).limit(3))

In [ ]:
# Przykład 1.1 - Bronze Layer (część 3: zapis do Delta table)

# Zapisz do Bronze schema
(
    orders_bronze
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(bronze_table)
)

print(f"✅ Utworzono Bronze table: {bronze_table}")
print(f"Liczba rekordów: {spark.table(bronze_table).count()}")

# Weryfikacja danych
print("\n=== Pełny rekord z Bronze ===")
display(spark.table(bronze_table).limit(2))

**Wyjaśnienie:**

Bronze layer:
- **Immutable**: Dane zapisane "as-is" bez modyfikacji wartości biznesowych
- **Audit trail**: Każdy rekord ma metadata: kiedy, skąd, przez kogo został załadowany
- **Append-only**: Idealnie nadaje się do incremental loads z COPY INTO lub Auto Loader
- **Retention**: Często długa (lata) jako źródło prawdy do reprocessingu

**Struktura danych źródłowych (orders_batch.json)**:
- `order_id`: String (np. "ORD00000001")
- `customer_id`: String (np. "CUST005909")
- `order_datetime`: Timestamp (np. "2024-12-31T23:56:00")
- `total_amount`: Double - całkowita wartość zamówienia
- `payment_method`: String - metoda płatności

### Przykład 1.2: Silver Layer - Cleansed & Validated

**Cel:** Transformacja Bronze → Silver z data quality checks

In [0]:
# Przykład 1.2 - Silver Layer (część 1: deduplikacja i walidacja NOT NULL)

spark.sql(f"USE SCHEMA {SILVER_SCHEMA}")

silver_table = f"{SILVER_SCHEMA}.orders_silver"

# Wczytaj dane z Bronze
orders_bronze_df = spark.table(bronze_table)

print(f"=== Bronze input ===")
print(f"Liczba rekordów: {orders_bronze_df.count()}")

# Deduplikacja po kluczu biznesowym
orders_deduped = orders_bronze_df.dropDuplicates(["order_id"])
print(f"\n✓ Po deduplikacji: {orders_deduped.count()} rekordów")

# Walidacja: usuń rekordy z NULL w kluczowych kolumnach
orders_validated = (
    orders_deduped
    .filter(F.col("order_id").isNotNull())
    .filter(F.col("customer_id").isNotNull())
    .filter(F.col("product_id").isNotNull())
)

print(f"✓ Po walidacji NOT NULL: {orders_validated.count()} rekordów")

In [ ]:
# Przykład 1.2 - Silver Layer (część 2: walidacja biznesowa i standaryzacja)

# Walidacja biznesowa: total_amount > 0
orders_business_validated = (
    orders_validated
    .filter(F.col("total_amount") > 0)
)

print(f"=== Walidacja biznesowa ===")
print(f"✓ Po walidacji total_amount > 0: {orders_business_validated.count()} rekordów")

# Standaryzacja dat i tekstów
orders_standardized = (
    orders_business_validated
    
    # Standaryzacja dat - użyj order_datetime zamiast order_date
    .withColumn("order_date", F.to_date(F.col("order_datetime")))
    .withColumn("order_timestamp", F.to_timestamp(F.col("order_datetime")))
    
    # Standaryzacja tekstów
    .withColumn("payment_method", F.upper(F.trim(F.col("payment_method"))))
    
    # Type casting
    .withColumn("total_amount", F.col("total_amount").cast("decimal(10,2)"))
    
    # Derived column: order_status (business logic example)
    # COMPLETED: positive amounts (normal transactions)
    # (W praktyce, status mógłby być wyciągnięty z systemu źródłowego)
    .withColumn("order_status", 
                F.when(F.col("total_amount") > 0, "COMPLETED")
                .otherwise("UNKNOWN"))
)

print("\n=== Silver transformations complete (with derived columns) ===")
display(orders_standardized.select(
    "order_id", 
    "order_date", 
    "order_timestamp",
    "total_amount", 
    "payment_method",
    "order_status"  # New derived column
).limit(3))

In [ ]:
# Przykład 1.2 - Silver Layer (część 3: Silver metadata i zapis)

# Dodaj Silver metadata
orders_silver = (
    orders_standardized
    .withColumn("_silver_processed_timestamp", F.current_timestamp())
    .withColumn("_data_quality_flag", F.lit("VALID"))
)

# Quality metrics
bronze_count = orders_bronze_df.count()
silver_count = orders_silver.count()
rejected_count = bronze_count - silver_count
rejection_rate = (rejected_count / bronze_count * 100) if bronze_count > 0 else 0

print("=" * 70)
print("SILVER LAYER - DATA QUALITY METRICS")
print("=" * 70)
print(f"Bronze input:  {bronze_count:,} records")
print(f"Silver output: {silver_count:,} records")
print(f"Rejected:      {rejected_count:,} records ({rejection_rate:.2f}%)")
print("=" * 70)

# Zapisz do Silver schema
(
    orders_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(silver_table)
)

print(f"\n✅ Utworzono Silver table: {silver_table}")
display(spark.table(silver_table).limit(3))

**Wyjaśnienie:**

Silver layer:
- **Data Quality**: Walidacja biznesowa (total_amount > 0), walidacja schematów (NOT NULL)
- **Deduplikacja**: Usunięcie duplikatów po kluczu biznesowym
- **Standardizacja**: Ujednolicenie formatów (daty, teksty, case sensitivity)
- **Derived Columns**: Tworzenie nowych kolumn na podstawie logiki biznesowej (np. `order_status` z `total_amount`)
- **Incremental friendly**: Można używać MERGE dla slowly changing dimensions

**Nota**: W tym przykładzie:
1. Transformujemy `order_datetime` (timestamp) na `order_date` (date) dla łatwiejszego partycjonowania
2. Dodajemy derived column `order_status = COMPLETED` (w rzeczywistych scenariuszach status mógłby być wyciągany z systemu źródłowego lub obliczany na podstawie bardziej złożonej logiki biznesowej)
3. Silver Layer jest miejscem, gdzie dodajemy business context do surowych danych

---

## Sekcja 1.4: Data Quality w kontekście warstw

**Cel:** Zrozumienie jak data quality gates są implementowane na każdym etapie Medallion Architecture.

---

### Data Quality per Layer

```
┌──────────────────────────────────────────────────────────────┐
│                    BRONZE LAYER                               │
│  Quality Focus: Completeness & Audit                         │
├──────────────────────────────────────────────────────────────┤
│  ✅ Checks:                                                   │
│  • File successfully loaded?                                  │
│  • All expected files arrived?                               │
│  • Audit metadata present?                                    │
│                                                               │
│  ⚠️  NO Business Validation (raw = raw)                      │
└───────────────────────┬──────────────────────────────────────┘
                        │ Quality Gate 1
                        ▼
┌──────────────────────────────────────────────────────────────┐
│                    SILVER LAYER                               │
│  Quality Focus: Validity & Consistency                       │
├──────────────────────────────────────────────────────────────┤
│  ✅ Checks:                                                   │
│  • Schema validation (NOT NULL, data types)                  │
│  • Business rules (amount > 0, valid dates)                  │
│  • Deduplikacja (unique business keys)                       │
│  • Referential integrity (FK checks)                         │
│  • Data standarization (formats, case, trim)                 │
│                                                               │
│  📊 Metrics:                                                  │
│  • Rejection rate (Bronze → Silver)                          │
│  • Invalid records count                                     │
│  • Data quality score per column                             │
└───────────────────────┬──────────────────────────────────────┘
                        │ Quality Gate 2
                        ▼
┌──────────────────────────────────────────────────────────────┐
│                    GOLD LAYER                                 │
│  Quality Focus: Completeness & Business Logic                │
├──────────────────────────────────────────────────────────────┤
│  ✅ Checks:                                                   │
│  • Aggregate completeness (all partitions?)                  │
│  • KPI calculations correct?                                 │
│  • Dimension joins successful (no orphans)?                  │
│  • Business metric thresholds met?                           │
│                                                               │
│  📊 Metrics:                                                  │
│  • Unmatched dimension records                               │
│  • KPI variance vs expected                                  │
│  • Data freshness (last update time)                         │
└──────────────────────────────────────────────────────────────┘
```

---

### Data Quality Framework

**1. Bronze Layer Quality**

**Focus:** Audit trail i completeness

```python
# Bronze quality checks
bronze_quality_checks = {
    "file_loaded": spark.table(bronze_table).count() > 0,
    "audit_columns_present": all(col in spark.table(bronze_table).columns 
                                  for col in ["_bronze_ingest_timestamp", "_bronze_source_file"]),
    "no_duplicates_in_batch": spark.table(bronze_table)
                                .groupBy("_bronze_source_file").count()
                                .filter("count > 1").count() == 0
}
```

**Key Metrics:**
- Files loaded successfully
- Records per file
- Load time per file
- Schema drift detection

---

**2. Silver Layer Quality**

**Focus:** Validity, consistency, standardization

```python
# Silver quality checks
silver_quality_checks = {
    "not_null": {
        "order_id": orders_silver.filter(F.col("order_id").isNull()).count() == 0,
        "customer_id": orders_silver.filter(F.col("customer_id").isNull()).count() == 0
    },
    "business_rules": {
        "positive_amount": orders_silver.filter(F.col("total_amount") <= 0).count() == 0,
        "valid_dates": orders_silver.filter(F.col("order_date") > F.current_date()).count() == 0
    },
    "uniqueness": {
        "unique_order_ids": orders_silver.select("order_id").distinct().count() == orders_silver.count()
    }
}
```

**Key Metrics:**
- **Rejection rate**: (Bronze_count - Silver_count) / Bronze_count
- **Null percentage** per kolumna
- **Invalid records** per validation rule
- **Duplicate rate**
- **Standardization coverage** (% records standardized)

---

**3. Gold Layer Quality**

**Focus:** Business logic correctness, completeness

```python
# Gold quality checks
gold_quality_checks = {
    "completeness": {
        "all_dates_present": check_date_continuity(gold_daily_summary),
        "no_missing_partitions": check_partition_completeness(gold_table)
    },
    "referential_integrity": {
        "orphan_customers": check_unmatched_fk(gold_fact, dim_customer, "customer_id"),
        "orphan_products": check_unmatched_fk(gold_fact, dim_product, "product_id")
    },
    "business_logic": {
        "revenue_matches": sum(gold_fact.total_amount) == sum(silver.total_amount),
        "kpi_thresholds": check_kpi_thresholds(gold_kpis)
    }
}
```

**Key Metrics:**
- **Unmatched foreign keys** (orphans)
- **Aggregate variance** (Gold vs Silver source)
- **Data freshness** (time since last refresh)
- **KPI anomalies** (unexpected spikes/drops)

---

### Quality Monitoring Dashboard

**Przykład: Real-time quality monitoring**

```python
# Quality dashboard per layer
quality_dashboard = {
    "Bronze": {
        "total_records": bronze_count,
        "unique_files": bronze_df.select("_bronze_source_file").distinct().count(),
        "load_timestamp": bronze_df.agg(F.max("_bronze_ingest_timestamp")).collect()[0][0]
    },
    "Silver": {
        "total_records": silver_count,
        "rejection_rate": rejection_rate,
        "null_percentage": calculate_null_percentage(silver_df),
        "quality_flag_distribution": silver_df.groupBy("_data_quality_flag").count().collect()
    },
    "Gold": {
        "total_records": gold_count,
        "orphan_rate": calculate_orphan_rate(gold_fact),
        "freshness_hours": calculate_freshness_hours(gold_df)
    }
}
```

---

### Alerting Thresholds

**Ustaw alerty dla anomalii w data quality:**

| Metric | Threshold | Action |
|--------|-----------|--------|
| Bronze rejection rate | > 5% | ⚠️ Warning |
| Bronze rejection rate | > 10% | ❌ Critical - Stop pipeline |
| Silver null % | > 2% | ⚠️ Warning |
| Gold orphan rate | > 1% | ⚠️ Warning - Check dimension loads |
| Data freshness | > 24h | ⚠️ Warning - Pipeline delayed |

---

### Best Practices

**✅ Bronze Quality:**
- Monitor file arrival patterns (missing files?)
- Track schema evolution (new columns?)
- Audit trail completeness

**✅ Silver Quality:**
- Define clear validation rules per column
- Quarantine invalid records (don't drop silently)
- Monitor rejection rate trends
- Alert on sudden spikes in rejections

**✅ Gold Quality:**
- Validate aggregate completeness (all partitions?)
- Check referential integrity (no orphans)
- Monitor KPI variance (expected ranges?)
- Track data freshness (SLA compliance)

### Przykład 1.3: Gold Layer - Business Aggregates

**Cel:** Utworzenie Gold layer z KPI dla analityki i raportowania

In [ ]:
# Przykład 1.3 - Gold Layer (część 1: Wczytanie danych z Silver + data quality check)

spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

# Wczytaj dane z Silver
orders_silver_df = spark.table(silver_table)

print("=" * 70)
print("GOLD LAYER - DATA QUALITY CHECK (COMPLETENESS)")
print("=" * 70)

# Data quality check: Completeness
silver_count = orders_silver_df.count()
silver_date_range = orders_silver_df.agg(
    F.min("order_date").alias("min_date"),
    F.max("order_date").alias("max_date")
).collect()[0]

print(f"Silver records: {silver_count:,}")
print(f"Date range: {silver_date_range['min_date']} to {silver_date_range['max_date']}")

# Check date continuity (czy wszystkie dni są obecne?)
expected_days = (silver_date_range['max_date'] - silver_date_range['min_date']).days + 1
actual_days = orders_silver_df.select("order_date").distinct().count()

print(f"\nExpected days: {expected_days}")
print(f"Actual days: {actual_days}")
if expected_days == actual_days:
    print("✅ Date continuity: OK (no missing dates)")
else:
    print(f"⚠️ Date continuity: INCOMPLETE (missing {expected_days - actual_days} days)")

# Check for null critical columns
null_checks = {
    "order_id": orders_silver_df.filter(F.col("order_id").isNull()).count(),
    "customer_id": orders_silver_df.filter(F.col("customer_id").isNull()).count(),
    "total_amount": orders_silver_df.filter(F.col("total_amount").isNull()).count(),
    "order_date": orders_silver_df.filter(F.col("order_date").isNull()).count()
}

print("\n[NULL CHECKS]")
all_passed = True
for col_name, null_count in null_checks.items():
    status = "✅" if null_count == 0 else "❌"
    print(f"{status} {col_name}: {null_count} nulls")
    if null_count > 0:
        all_passed = False

if all_passed:
    print("\n✅ All data quality checks PASSED - proceeding to Gold aggregation")
else:
    print("\n⚠️ Some quality checks FAILED - review Silver layer before Gold aggregation")
print("=" * 70)

In [ ]:
# Przykład 1.3 - Gold Layer (część 2: Agregacja biznesowa + KPI calculation)

print("=" * 70)
print("GOLD LAYER - BUSINESS AGGREGATION & KPI CALCULATION")
print("=" * 70)

# Gold aggregation: Daily order summary z KPI
daily_summary = (
    orders_silver_df
    .groupBy("order_date", "order_status")
    .agg(
        # Volume metrics
        F.count("order_id").alias("total_orders"),
        F.countDistinct("customer_id").alias("unique_customers"),
        
        # Revenue metrics
        F.sum("total_amount").alias("total_revenue"),
        F.avg("total_amount").alias("avg_order_value"),
        F.min("total_amount").alias("min_order_value"),
        F.max("total_amount").alias("max_order_value"),
        
        # Payment method distribution
        F.count(F.when(F.col("payment_method") == "credit_card", 1)).alias("credit_card_orders"),
        F.count(F.when(F.col("payment_method") == "paypal", 1)).alias("paypal_orders"),
        F.count(F.when(F.col("payment_method") == "bank_transfer", 1)).alias("bank_transfer_orders")
    )
    .orderBy("order_date", "order_status")
)

# Calculate derived KPIs
daily_summary_with_kpis = (
    daily_summary
    .withColumn("revenue_per_customer", 
                F.round(F.col("total_revenue") / F.col("unique_customers"), 2))
    .withColumn("credit_card_penetration_pct",
                F.round(F.col("credit_card_orders") / F.col("total_orders") * 100, 2))
)

print("\n[GOLD AGGREGATION PREVIEW]")
display(daily_summary_with_kpis.limit(5))

# Summary statistics
total_days = daily_summary_with_kpis.select("order_date").distinct().count()
total_orders_gold = daily_summary_with_kpis.agg(F.sum("total_orders")).collect()[0][0]
total_revenue_gold = daily_summary_with_kpis.agg(F.sum("total_revenue")).collect()[0][0]

print(f"\n[GOLD SUMMARY STATISTICS]")
print(f"Total days aggregated: {total_days}")
print(f"Total orders (all days): {total_orders_gold:,}")
print(f"Total revenue (all days): ${total_revenue_gold:,.2f}")
print("=" * 70)

In [ ]:
# Przykład 1.3 - Gold Layer (część 3: Gold audit metadata + zapis do Delta)

print("=" * 70)
print("GOLD LAYER - AUDIT METADATA & DELTA WRITE")
print("=" * 70)

# Dodaj Gold audit metadata
gold_table = f"{GOLD_SCHEMA}.daily_order_summary"

daily_summary_final = (
    daily_summary_with_kpis
    .withColumn("_gold_created_timestamp", F.current_timestamp())
    .withColumn("_gold_aggregation_level", F.lit("DAILY"))
    .withColumn("_gold_source_silver_table", F.lit(silver_table))
    .withColumn("_gold_refresh_id", F.lit(str(uuid.uuid4())))
)

# Zapisz do Gold schema
(
    daily_summary_final
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(gold_table)
)

print(f"\n✅ Utworzono Gold table: {gold_table}")

# Verify Gold table
gold_verification = spark.table(gold_table)
gold_count = gold_verification.count()

print(f"\n[GOLD TABLE VERIFICATION]")
print(f"Gold records: {gold_count}")
print(f"Gold schema columns: {len(gold_verification.columns)}")

# Gold lineage tracking
print(f"\n[GOLD LINEAGE]")
print(f"Source: {silver_table}")
print(f"Target: {gold_table}")
print(f"Aggregation level: DAILY")
print(f"Aggregation keys: order_date, order_status")
print(f"Created at: {daily_summary_final.select('_gold_created_timestamp').first()[0]}")

# Display sample Gold records with audit metadata
print("\n[GOLD TABLE PREVIEW - with audit metadata]")
display(gold_verification.select(
    "order_date",
    "order_status",
    "total_orders",
    "total_revenue",
    "unique_customers",
    "_gold_created_timestamp",
    "_gold_aggregation_level"
).limit(5))

print("=" * 70)

**Wyjaśnienie:**

Gold layer:
- **Business-level**: KPI i metryki zgodne z business definitions
- **Denormalized**: Często szeroka tabela z joinami już wykonanymi (performance dla BI)
- **Aggregated**: Dane pre-aggregowane (daily, weekly, monthly) dla szybkich dashboardów
- **BI-ready**: Bezpośrednie źródło dla Power BI, Tableau, Looker

**Nota**: W tym przykładzie agregujemy zamówienia po dacie i metodzie płatności (payment_method). W rzeczywistych scenariuszach możesz agregować po innych wymiarach biznesowych jak region, kategoria produktu, czy segment klienta.

---

### Gold Layer - Monitoring & Lineage

**Dlaczego monitoring Gold layer jest krytyczny?**

Gold layer to końcowy produkt dla biznesu - dashboardy, raporty, modele ML. Problemy w Gold = bezpośredni wpływ na decyzje biznesowe.

---

**Kluczowe metryki Gold Layer:**

**1. Data Freshness**
```python
# Sprawdź jak świeże są dane w Gold
gold_freshness = (
    spark.table(gold_table)
    .agg(F.max("_gold_created_timestamp").alias("last_refresh"))
    .collect()[0][0]
)

hours_since_refresh = (datetime.now() - gold_freshness).total_seconds() / 3600

if hours_since_refresh > 24:
    print(f"⚠️ Gold layer is {hours_since_refresh:.1f} hours old - SLA breach!")
else:
    print(f"✅ Gold layer is fresh ({hours_since_refresh:.1f} hours old)")
```

**2. Aggregate Completeness**
```python
# Czy wszystkie dni są zagregowane?
from datetime import timedelta

max_silver_date = spark.table(silver_table).agg(F.max("order_date")).collect()[0][0]
max_gold_date = spark.table(gold_table).agg(F.max("order_date")).collect()[0][0]

if max_gold_date < max_silver_date:
    missing_days = (max_silver_date - max_gold_date).days
    print(f"⚠️ Gold is behind Silver by {missing_days} days")
else:
    print(f"✅ Gold is up-to-date with Silver")
```

**3. Revenue Reconciliation (Gold vs Silver)**
```python
# Gold revenue musi się zgadzać z Silver revenue
silver_revenue = spark.table(silver_table).agg(F.sum("total_amount")).collect()[0][0]
gold_revenue = spark.table(gold_table).agg(F.sum("total_revenue")).collect()[0][0]

variance_pct = abs((gold_revenue - silver_revenue) / silver_revenue * 100)

if variance_pct > 0.01:  # 0.01% tolerance
    print(f"❌ Revenue mismatch: {variance_pct:.4f}% variance!")
else:
    print(f"✅ Revenue reconciled: {variance_pct:.4f}% variance")
```

**4. Unmatched Foreign Keys (Orphans)**
```python
# Przykład: Sprawdź czy wszystkie customer_id z Gold mają odpowiednik w dim_customer
# (zakładając, że mamy dimension table)

# orphan_customers = (
#     gold_fact
#     .join(dim_customer, "customer_id", "left_anti")
#     .count()
# )
# 
# if orphan_customers > 0:
#     print(f"⚠️ {orphan_customers} orphan customers in Gold")
```

---

**Gold Layer Lineage Tracking:**

Gold layer często łączy wiele Silver tables (joins). Ważne jest śledzenie, które Silver tables były użyte:

```python
# Przykład: Multi-source Gold table
gold_with_lineage = (
    orders_silver
    .join(customers_silver, "customer_id")
    .join(products_silver, "product_id")
    .groupBy("order_date")
    .agg(...)
    .withColumn("_gold_source_tables", F.lit([
        "silver.orders",
        "silver.customers", 
        "silver.products"
    ]))
)
```

Dzięki temu wiemy: jeśli `silver.customers` ma problem, które Gold tables są affected.

---

## Sekcja 1.5: Audyt i Lineage - Metadane w każdym kroku

**Cel:** Zrozumienie jak implementować audit trail i data lineage w Medallion Architecture.

---

### Czym jest Data Lineage?

**Data Lineage** to śledzenie przepływu danych od źródła do destination, w tym wszystkie transformacje, filtry i agregacje po drodze.

```
Source System → Bronze → Silver → Gold → BI Dashboard
     │            │        │        │          │
     └─ When?     └─ Who?  └─ What? └─ How?  └─ Used by?
```

**Dlaczego lineage jest kluczowy?**
- **Compliance** (GDPR, SOX): "Where did this data come from?"
- **Troubleshooting**: "Why is this metric wrong?"
- **Impact Analysis**: "What breaks if I change this table?"
- **Data Quality**: "Which source file had bad data?"

---

### Audit Metadata per Layer

**Bronze Layer Audit Columns:**

```python
bronze_audit_columns = [
    "_bronze_ingest_timestamp",    # Kiedy dane trafiły do Bronze
    "_bronze_source_file",          # Skąd pochodzą (file path)
    "_bronze_ingested_by",          # Kto/co załadowało (user/job ID)
    "_bronze_version",              # Wersja schematu/procesu
    "_bronze_batch_id"              # UUID dla batch tracking (optional)
]
```

**Silver Layer Audit Columns:**

```python
silver_audit_columns = [
    "_silver_processed_timestamp",  # Kiedy Bronze → Silver
    "_data_quality_flag",           # VALID / INVALID / QUARANTINE
    "_silver_transformation_id",    # ID pipeline'u który przetworzył
    "_silver_source_bronze_version" # Z której wersji Bronze
]
```

**Gold Layer Audit Columns:**

```python
gold_audit_columns = [
    "_gold_created_timestamp",      # Kiedy Silver → Gold
    "_gold_refresh_timestamp",      # Ostatni refresh (dla aggregates)
    "_gold_aggregation_level",      # DAILY / MONTHLY / YEARLY
    "_gold_source_silver_tables"    # Lista Silver tables użytych w aggregation
]
```

---

### Implementacja Audit Trail

**Przykład: Tracking kompletnego pipeline'u**

In [ ]:
# Przykład 1.5 - Audit Trail Tracking

import uuid

# Generate unique batch ID
batch_id = str(uuid.uuid4())

print("=" * 70)
print("AUDIT TRAIL TRACKING - END-TO-END LINEAGE")
print("=" * 70)
print(f"\nBatch ID: {batch_id}")

# Bronze: Track source
bronze_lineage = spark.table(bronze_table).select(
    "_bronze_ingest_timestamp",
    "_bronze_source_file",
    "_bronze_ingested_by",
    "_bronze_version"
).distinct()

print("\n[BRONZE LAYER - Source Tracking]")
display(bronze_lineage)

# Silver: Track transformation
silver_lineage = spark.table(silver_table).select(
    "_silver_processed_timestamp",
    "_data_quality_flag"
).distinct()

print("\n[SILVER LAYER - Transformation Tracking]")
display(silver_lineage)

# Lineage query: Które Bronze files stworzyły które Silver records?
bronze_to_silver_lineage = (
    spark.table(silver_table)
    .join(
        spark.table(bronze_table),
        ["order_id"],
        "inner"
    )
    .select(
        F.col("order_id"),
        F.col("_bronze_source_file").alias("source_file"),
        F.col("_bronze_ingest_timestamp").alias("bronze_loaded_at"),
        F.col("_silver_processed_timestamp").alias("silver_processed_at")
    )
    .distinct()
)

print("\n[LINEAGE - Bronze → Silver Tracking]")
display(bronze_to_silver_lineage.limit(10))

# Calculate processing lag
processing_lag = (
    bronze_to_silver_lineage
    .withColumn(
        "processing_lag_seconds",
        F.unix_timestamp("silver_processed_at") - F.unix_timestamp("bronze_loaded_at")
    )
    .agg(
        F.avg("processing_lag_seconds").alias("avg_lag_seconds"),
        F.max("processing_lag_seconds").alias("max_lag_seconds")
    )
)

print("\n[PROCESSING METRICS]")
display(processing_lag)

---

## Sekcja 2: ETL vs ELT Approach

**Wprowadzenie teoretyczne:**

Medallion architecture wspiera ELT (Extract-Load-Transform) approach, w przeciwieństwie do tradycyjnego ETL. Dane są najpierw ładowane do Bronze (Load), a potem transformowane w Silver/Gold (Transform).

**Kluczowe różnice:**
- **ETL**: Transform before load - dane są czyszczone poza data warehouse
- **ELT**: Load then transform - surowe dane w Bronze, transformacje w lakehouse
- **Zalety ELT**: Możliwość reprocessingu, data lineage, audit trail, schema evolution

---

## Sekcja 2.5: Zasady projektowania Pipeline'ów

**Cel:** Zrozumienie best practices przy projektowaniu production-ready Medallion pipelines.

---

### 10 Zasad Projektowania Pipeline'ów

**1. Idempotency (Powtarzalność)**

Pipeline musi dać ten sam wynik przy wielokrotnym uruchomieniu na tych samych danych.

```python
# ✅ DOBRZE: Idempotent (overwrite lub MERGE z kluczem)
df.write.mode("overwrite").saveAsTable(table_name)

# ❌ ŹLE: Non-idempotent (append bez deduplikacji)
df.write.mode("append").saveAsTable(table_name)  # Duplikaty przy retry!
```

**Best Practice:**
- Bronze: COPY INTO (automatic idempotency)
- Silver: MERGE na business key
- Gold: Overwrite partition lub MERGE

---

**2. Incrementality (Przetwarzanie przyrostowe)**

Procesuj tylko nowe/zmienione dane, nie całą tabelę.

```python
# ✅ DOBRZE: Incremental processing
max_processed_ts = spark.table(silver_table).agg(F.max("_bronze_ingest_timestamp")).collect()[0][0]
new_bronze = spark.table(bronze_table).filter(F.col("_bronze_ingest_timestamp") > max_processed_ts)

# ❌ ŹLE: Full table scan każdorazowo
all_bronze = spark.table(bronze_table)  # Procesuje wszystko od nowa
```

**Best Practice:**
- Watermarking: Track last processed timestamp
- Partitioning: Proces tylko affected partitions
- Change Data Capture (CDC): Track tylko changed records

---

**3. Separation of Concerns (Izolacja warstw)**

Każda warstwa ma określone responsibility.

```
Bronze:  Data Ingestion + Audit
         ↓
Silver:  Data Quality + Standardization
         ↓
Gold:    Business Logic + Aggregation
```

**Best Practice:**
- Nie mieszaj business logic w Bronze (raw = raw)
- Nie rób agregacji w Silver (tylko cleaning)
- Nie rób walidacji w Gold (już zwalidowane w Silver)

---

**4. Schema Evolution (Obsługa zmian schematu)**

Pipeline musi obsługiwać nowe kolumny bez breaking.

```python
# ✅ DOBRZE: Merge schema
df.write.option("mergeSchema", "true").mode("append").saveAsTable(table_name)

# ✅ DOBRZE: Schema validation
if new_col in df.columns:
    # Process new column
    pass
```

**Best Practice:**
- Bronze: Allow all schema changes (permissive mode)
- Silver: Validate schema against expected (fail on breaking changes)
- Gold: Fixed schema (breaking changes require re-architecture)

---

**5. Error Handling & Quarantine**

Nie fail całego pipeline'u przez kilka bad records.

```python
# ✅ DOBRZE: Quarantine invalid records
valid_records = df.filter(validation_condition)
invalid_records = df.filter(~validation_condition)

invalid_records.write.mode("append").saveAsTable("quarantine_table")
valid_records.write.mode("append").saveAsTable("silver_table")
```

**Best Practice:**
- Log invalid records do quarantine table
- Alert on spike w rejection rate
- Manual review quarantine periodically

---

**6. Monitoring & Observability**

Co nie jest monitorowane, nie istnieje.

```python
# Track key metrics
metrics = {
    "records_processed": df.count(),
    "processing_time_seconds": end_time - start_time,
    "rejection_rate": rejected_count / total_count,
    "pipeline_status": "SUCCESS" if no_errors else "FAILED"
}

# Log do monitoring system
log_metrics(metrics, table_name, timestamp)
```

**Best Practice:**
- Monitor per layer: Bronze count, Silver rejection, Gold freshness
- Alert thresholds: rejection > 5%, lag > 1h
- Dashboard per pipeline: throughput, latency, errors

---

**7. Partitioning Strategy**

Partycjonuj tylko jeśli tabela > 1TB i partition > 1GB.

```python
# ✅ DOBRZE: Partitioning po często filtrowanej kolumnie
df.write.partitionBy("order_date").saveAsTable(table_name)

# ❌ ŹLE: Over-partitioning (< 1GB partitions)
df.write.partitionBy("order_date", "order_hour", "customer_id").saveAsTable(table_name)
```

**Best Practice:**
- Bronze: Rzadko partycjonuj (bulk operations)
- Silver: Partition po date lub region
- Gold: Partition wg query patterns (date, region, category)

---

**8. Retry & Resilience**

Pipeline musi recovery z failures.

```python
# ✅ DOBRZE: Checkpoint dla streaming
(
    spark.readStream
    .option("checkpointLocation", checkpoint_path)
    .table(source_table)
    .writeStream
    .table(target_table)
)
```

**Best Practice:**
- Checkpoint locations dla streaming
- Exponential backoff dla retries
- Transaction logs (Delta) dla consistency

---

**9. Data Retention & Lifecycle**

Różne retention policies per warstwa.

```
Bronze:  3-7 lat   (compliance, reprocessing)
Silver:  1-2 lata  (operational analytics)
Gold:    6-12 mies (refreshable from Silver)
```

**Best Practice:**
- VACUUM starsze partycje regularnie
- Archive do cold storage (S3 Glacier, Azure Archive)
- Automated lifecycle policies (TTL)

---

**10. Testing & Validation**

Test pipeline przed production.

```python
# Unit tests
def test_silver_transformation():
    input_df = create_test_bronze_data()
    output_df = silver_transformation(input_df)
    assert output_df.filter(F.col("total_amount") <= 0).count() == 0

# Integration tests
def test_end_to_end_pipeline():
    run_pipeline(test_data_path)
    assert_gold_count_matches_expected()
```

**Best Practice:**
- Unit tests per transformation
- Integration tests end-to-end
- Data quality tests (Great Expectations, Deequ)
- Smoke tests w production (sanity checks)

---

### Pipeline Design Checklist

Przed deploymentem production pipeline, sprawdź:

- [ ] ✅ Idempotent? (retry-safe)
- [ ] ✅ Incremental? (tylko nowe dane)
- [ ] ✅ Schema evolution? (handle new columns)
- [ ] ✅ Error handling? (quarantine invalid)
- [ ] ✅ Monitoring? (metrics + alerts)
- [ ] ✅ Partitioning? (jeśli > 1TB)
- [ ] ✅ Checkpoint? (dla streaming)
- [ ] ✅ Retention policy? (lifecycle management)
- [ ] ✅ Tests? (unit + integration)
- [ ] ✅ Documentation? (README, lineage diagram)

### Przykład 2.1: ELT Pipeline - Incremental Processing

**Cel:** Demonstracja incremental ELT: nowe dane w Bronze → automatyczna propagacja do Silver/Gold

In [0]:
# Przykład 2.1 - Incremental ELT

# Symulacja: nowe dane przychodzą do Bronze
new_orders_data = [
    ("ORD99990001", "CUST009901", "PROD000001", "STORE001", "2025-01-20T10:00:00", 2, 175.00, 0, 350.00, "Credit Card"),
    ("ORD99990002", "CUST009902", "PROD000002", "STORE002", "2025-01-20T11:00:00", 1, 120.50, 0, 120.50, "Debit Card"),
    ("ORD99990003", "CUST009903", "PROD000003", "STORE003", "2025-01-21T09:00:00", 1, 499.99, 0, 499.99, "PayPal")
]

new_orders_df = spark.createDataFrame(
    new_orders_data,
    ["order_id", "customer_id", "product_id", "store_id", "order_datetime", "quantity", "unit_price", "discount_percent", "total_amount", "payment_method"]
)

# Dodaj audit metadata (Bronze standard)
new_orders_bronze = (
    new_orders_df
    .withColumn("_bronze_ingest_timestamp", F.current_timestamp())
    .withColumn("_bronze_source_file", F.lit("incremental_batch_2"))
    .withColumn("_bronze_ingested_by", F.lit(raw_user))
    .withColumn("_bronze_version", F.lit(2))
)

# Append do Bronze (ELT: Load first)
(
    new_orders_bronze
    .write
    .format("delta")
    .mode("append")
    .saveAsTable(bronze_table)
)

print(f"✓ Dodano {new_orders_df.count()} nowych rekordów do Bronze")
print(f"Bronze total: {spark.table(bronze_table).count()} records")

# Incremental Silver processing: tylko nowe Bronze records (version 2)
new_bronze_records = (
    spark.table(bronze_table)
    .filter(F.col("_bronze_version") == 2)
)

# Apply Silver transformations (zgodnie z komórką Silver Layer)
new_silver_records = (
    new_bronze_records
    .dropDuplicates(["order_id"])
    .filter(F.col("order_id").isNotNull())
    .filter(F.col("customer_id").isNotNull())
    .filter(F.col("total_amount") > 0)
    .withColumn("order_date", F.to_date(F.col("order_datetime")))
    .withColumn("order_timestamp", F.to_timestamp(F.col("order_datetime")))
    .withColumn("payment_method", F.upper(F.trim(F.col("payment_method"))))
    .withColumn("total_amount", F.col("total_amount").cast("decimal(10,2)"))
    .withColumn("order_status", F.when(F.col("total_amount") > 0, "COMPLETED").otherwise("UNKNOWN"))
    .withColumn("_silver_processed_timestamp", F.current_timestamp())
    .withColumn("_data_quality_flag", F.lit("VALID"))
)

# Append do Silver
(
    new_silver_records
    .write
    .format("delta")
    .mode("append")
    .saveAsTable(silver_table)
)

print(f"✓ Propagowano {new_silver_records.count()} rekordów do Silver")
print(f"Silver total: {spark.table(silver_table).count()} records")

# Gold: re-aggregate (lub incremental z MERGE)
# Dla uproszczenia: pełna re-agregacja po order_date + order_status
updated_daily_summary = (
    spark.table(silver_table)
    .groupBy("order_date", "order_status")
    .agg(
        F.count("order_id").alias("total_orders"),
        F.sum("total_amount").alias("total_revenue"),
        F.avg("total_amount").alias("avg_order_value"),
        F.min("total_amount").alias("min_order_value"),
        F.max("total_amount").alias("max_order_value"),
        F.countDistinct("customer_id").alias("unique_customers")
    )
    .withColumn("_gold_created_timestamp", F.current_timestamp())
    .withColumn("_gold_aggregation_level", F.lit("DAILY"))
)

(
    updated_daily_summary
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(gold_table)
)

print(f"✓ Zaktualizowano Gold layer")
print("\n=== Updated Gold Summary ===")
display(spark.table(gold_table).orderBy("order_date", "order_status"))

**Wyjaśnienie:**

Incremental ELT pattern:
1. **Bronze**: Append nowych danych z wersjonowaniem (bronze_version)
2. **Silver**: Proces tylko nowe Bronze records (watermark lub version)
3. **Gold**: Re-aggregate lub MERGE dla affected partitions

W produkcji: używamy Delta Live Tables lub Structured Streaming dla automatic incrementality.

---

### Przykład 2.2: Gold Incremental Refresh

**Cel:** Demonstracja incremental refresh dla Gold layer (zamiast full overwrite)

**Scenariusz:** Nowe dane w Silver tylko dla określonych dat - aktualizujemy tylko affected partitions w Gold.

In [ ]:
# Przykład 2.2 - Gold Incremental Refresh (MERGE INTO)

print("=" * 70)
print("GOLD INCREMENTAL REFRESH - MERGE INTO PATTERN")
print("=" * 70)

# Symulacja: nowe dane w Silver tylko dla 2025-01-20 i 2025-01-21
# (te daty zostały dodane w poprzednim przykładzie incremental ELT)

# Identyfikuj affected dates (dni, które mają nowe/zmienione dane w Silver)
affected_dates = ["2025-01-20", "2025-01-21"]

print(f"\n[AFFECTED DATES]")
print(f"Updating Gold for dates: {affected_dates}")

# Re-aggregate tylko dla affected dates
incremental_gold_updates = (
    spark.table(silver_table)
    .filter(F.col("order_date").isin(affected_dates))
    .groupBy("order_date", "order_status")
    .agg(
        F.count("order_id").alias("total_orders"),
        F.sum("total_amount").alias("total_revenue"),
        F.avg("total_amount").alias("avg_order_value"),
        F.min("total_amount").alias("min_order_value"),
        F.max("total_amount").alias("max_order_value"),
        F.countDistinct("customer_id").alias("unique_customers")
    )
    .withColumn("_gold_created_timestamp", F.current_timestamp())
    .withColumn("_gold_aggregation_level", F.lit("DAILY"))
    .withColumn("_gold_source_silver_table", F.lit(silver_table))
)

print(f"\n[INCREMENTAL UPDATES]")
print(f"Rows to update/insert: {incremental_gold_updates.count()}")
display(incremental_gold_updates)

# MERGE INTO Gold (update jeśli istnieje, insert jeśli nowe)
incremental_gold_updates.createOrReplaceTempView("gold_updates")

merge_sql = f"""
MERGE INTO {gold_table} AS target
USING gold_updates AS source
ON target.order_date = source.order_date 
   AND target.order_status = source.order_status
WHEN MATCHED THEN
  UPDATE SET
    target.total_orders = source.total_orders,
    target.total_revenue = source.total_revenue,
    target.avg_order_value = source.avg_order_value,
    target.min_order_value = source.min_order_value,
    target.max_order_value = source.max_order_value,
    target.unique_customers = source.unique_customers,
    target._gold_created_timestamp = source._gold_created_timestamp
WHEN NOT MATCHED THEN
  INSERT *
"""

result = spark.sql(merge_sql)

print(f"\n[MERGE RESULTS]")
print(f"✅ Gold table incrementally refreshed for {len(affected_dates)} dates")
print(f"Total Gold records now: {spark.table(gold_table).count()}")

# Verification: Check updated timestamps dla affected dates
updated_records = (
    spark.table(gold_table)
    .filter(F.col("order_date").isin(affected_dates))
    .select("order_date", "order_status", "total_orders", "_gold_created_timestamp")
)

print(f"\n[VERIFICATION - Updated Records]")
display(updated_records)
print("=" * 70)

**Wyjaśnienie:**

Gold Incremental Refresh pattern:
- **MERGE INTO**: Update dla existing dates, INSERT dla nowych
- **Performance**: Tylko affected partitions są przepisywane (partition pruning)
- **Idempotency**: Wielokrotne uruchomienie daje ten sam wynik
- **Production**: Używaj MERGE dla Gold daily refreshes zamiast full overwrite

**Alternatywa:** Partition overwrite
```python
# Zamiast MERGE, możesz użyć partition overwrite
(
    incremental_gold_updates
    .write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"order_date IN ({affected_dates_str})")
    .saveAsTable(gold_table)
)
```

**Trade-off:**
- MERGE: Precyzyjny (row-level), wolniejszy dla dużych partitions
- Partition Overwrite: Szybszy, ale przepisuje całą partycję

---

## Sekcja 3: Partitioning Strategy

**Wprowadzenie teoretyczne:**

Partycjonowanie to kluczowa decyzja architektoniczna w Medallion. Złe partycjonowanie prowadzi do small files problem lub nieefektywnych queries.

---

### Kiedy partycjonować?

**Zasada główna:** Partycjonuj **TYLKO** jeśli:
1. Tabela > 1 TB
2. Partition size > 1 GB
3. 80% queries filtruje po partitioning column

**Przykład:**

```python
# ✅ DOBRZE: Duża tabela (10 TB), 365 partycji = ~27 GB/partition
df.write.partitionBy("order_date").saveAsTable(large_table)

# ❌ ŹLE: Mała tabela (10 GB), 365 partycji = ~27 MB/partition (small files!)
df.write.partitionBy("order_date").saveAsTable(small_table)
```

---

### Partitioning per Layer

```
┌─────────────────────────────────────────────────────────────┐
│  BRONZE LAYER                                                │
│  Recommendation: NO PARTITIONING (lub minimal)               │
├─────────────────────────────────────────────────────────────┤
│  Dlaczego?                                                   │
│  • Append-only bulk operations (nie ma filtrów)             │
│  • Small files problem (każdy ingest = nowa partycja)       │
│  • Bulk reads dla Silver processing                          │
│                                                              │
│  Wyjątek: Jeśli > 10 TB, partition po ingest_date           │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│  SILVER LAYER                                                │
│  Recommendation: Partition po DATE lub REGION               │
├─────────────────────────────────────────────────────────────┤
│  Dlaczego?                                                   │
│  • Incremental MERGE (tylko affected partitions)            │
│  • Time-based queries (last 7 days, last month)             │
│  • Partition pruning dla Gold aggregations                   │
│                                                              │
│  Best Practice: Partition po order_date (daily granularity) │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│  GOLD LAYER                                                  │
│  Recommendation: Partition po Business Dimensions           │
├─────────────────────────────────────────────────────────────┤
│  Dlaczego?                                                   │
│  • BI queries filtrują po date, region, category            │
│  • Partition pruning = fast dashboard loads                 │
│  • Incremental refresh (tylko affected partitions)          │
│                                                              │
│  Best Practice: Partition po year + month (lub date)        │
└─────────────────────────────────────────────────────────────┘
```

---

### Partition Size Guidelines

**Cel:** Partition size 1-10 GB (sweet spot: 2-4 GB)

| Partition Size | Status | Action |
|----------------|--------|--------|
| < 100 MB | ❌ Too small | Increase partitioning granularity |
| 100 MB - 1 GB | ⚠️ Small | Consider coarser partitioning |
| 1 GB - 10 GB | ✅ Optimal | Good |
| > 10 GB | ⚠️ Large | Consider finer partitioning |

**Przykład kalkulacji:**

```
Tabela: 10 TB
Partitioning: order_date (daily)
Dane za: 2 lata = 730 dni

Partition size = 10 TB / 730 = ~13.7 GB/partition ⚠️ (trochę za duży)

Solution: Partition po order_date + region (5 regions)
Partition size = 10 TB / (730 * 5) = ~2.7 GB/partition ✅
```

---

### Partitioning Patterns

**Pattern 1: Date-based (najczęstszy)**

```python
# Daily partitions
df.write.partitionBy("order_date").saveAsTable(table_name)

# Monthly partitions (dla długich historii)
df = df.withColumn("year_month", F.date_format("order_date", "yyyy-MM"))
df.write.partitionBy("year_month").saveAsTable(table_name)

# Year + Month (hierarchical)
df = df.withColumn("year", F.year("order_date"))
df = df.withColumn("month", F.month("order_date"))
df.write.partitionBy("year", "month").saveAsTable(table_name)
```

**Pattern 2: Region-based**

```python
# Geographic partitioning
df.write.partitionBy("country").saveAsTable(table_name)

# Hierarchical: region → country
df.write.partitionBy("region", "country").saveAsTable(table_name)
```

**Pattern 3: Category-based**

```python
# Product category
df.write.partitionBy("product_category").saveAsTable(table_name)

# Multi-dimensional: date + category
df.write.partitionBy("order_date", "product_category").saveAsTable(table_name)
```

---

### Partition Pruning

**Partition pruning** = Spark czyta tylko partycje spełniające WHERE predicate.

```python
# ✅ DOBRZE: Partition pruning enabled
spark.table(partitioned_table).filter("order_date = '2025-01-20'")
# Spark czyta TYLKO partition order_date=2025-01-20 (fast!)

# ❌ ŹLE: No partition pruning (full table scan)
spark.table(partitioned_table).filter(F.dayofweek("order_date") == 1)
# Spark musi czytać WSZYSTKIE partycje (slow!)
```

**Best Practice:** Query patterns powinny filtrować po partitioning column.

---

### Small Files Problem

**Problem:** Setki małych plików w partycji (< 100 MB każdy)

**Przyczyny:**
- Częste małe zapisy (micro-batches)
- Over-partitioning (partition size < 1 GB)
- Brak compaction

**Rozwiązanie:**

```python
# 1. Auto Optimize (Databricks)
spark.sql(f"""
    ALTER TABLE {table_name} 
    SET TBLPROPERTIES (
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true'
    )
""")

# 2. Manual OPTIMIZE
spark.sql(f"OPTIMIZE {table_name}")

# 3. ZORDER BY (multi-dimensional queries)
spark.sql(f"OPTIMIZE {table_name} ZORDER BY (country, product_category)")
```

---

### Partitioning Anti-Patterns

**❌ Over-partitioning**

```python
# BAD: Partition size < 100 MB
df.write.partitionBy("order_date", "order_hour", "customer_id").saveAsTable(table_name)
# Result: Tysiące małych partycji
```

**❌ Partitioning po high-cardinality column**

```python
# BAD: 1M unique customer_ids = 1M partitions!
df.write.partitionBy("customer_id").saveAsTable(table_name)
```

**❌ Partitioning bez query pattern analysis**

```python
# BAD: Partitioning po kolumnie nigdy nie używanej w WHERE
df.write.partitionBy("internal_id").saveAsTable(table_name)
# Queries: SELECT * FROM table WHERE order_date = '...'  (no pruning!)
```

---

### Zasady partycjonowania

- **Bronze**: Rzadko partycjonujemy (append-only, bulk operations)
- **Silver**: Partycjonowanie po dacie lub region dla incremental MERGE
- **Gold**: Partycjonowanie wg wymiarów zapytań (date, region, product_category)
- **Reguła**: Partycjonuj tylko jeśli tabela > 1 TB i partition size > 1 GB
- **Query patterns**: 80% queries powinno filtrować po partitioning column

### Przykład 3.1: Partycjonowanie Silver layer po dacie

**Cel:** Demonstracja partitioned table dla efektywnych incremental updates

In [0]:
# Przykład 3.1 - Partitioned Silver table

# Utwórz Silver table z partycjonowaniem po order_date
silver_partitioned_table = f"{SILVER_SCHEMA}.orders_silver_partitioned"

(
    spark.table(silver_table)
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("order_date")
    .option("overwriteSchema", "true")
    .saveAsTable(silver_partitioned_table)
)

print(f"✓ Utworzono partycjonowaną tabelę: {silver_partitioned_table}")

# Sprawdź partycje
partitions = spark.sql(f"SHOW PARTITIONS {silver_partitioned_table}")
print("\n=== Partycje ===")
display(partitions)

# DESCRIBE DETAIL - sprawdź partitioning columns
detail = spark.sql(f"DESCRIBE DETAIL {silver_partitioned_table}")
print("\n=== Detail (partitionColumns) ===")
display(detail.select("name", "partitionColumns", "numFiles"))

**Wyjaśnienie:**

Partycjonowanie:
- **Partition pruning**: Spark czyta tylko partycje spełniające predicate (WHERE order_date = '2025-01-20')
- **Incremental MERGE**: UPDATE/DELETE tylko affected partitions
- **Trade-off**: Zbyt dużo partycji (< 1 GB) powoduje small files problem

Best practice: Partycjonuj po kolumnie używanej w 80% zapytań (często: date, region).

---

## Best Practices

**Projektowanie warstw:**
- **Bronze**: Immutable, append-only. Długa retention (lata). Audit metadata obowiązkowe.
- **Silver**: Idempotentne transformacje. Możliwość reprocessingu z Bronze. MERGE dla SCD.
- **Gold**: Denormalized, aggregated. Partition wg business dimensions. Krótka retention (miesięcy).

**Data Quality:**
- **Bronze → Silver**: Walidacja schematów, business rules, deduplikacja
- **Silver → Gold**: Sprawdzenie completeness (czy wszystkie Bronze records dotarły?)
- **Gold**: Revenue reconciliation, orphan checks, data freshness monitoring
- **Expectations**: Używaj Delta Live Tables expectations (warn/drop/fail)

**Performance:**
- **Partycjonowanie**: Tylko dla dużych tabel (>1TB), partition size > 1GB
- **ZORDER**: Silver/Gold - po kluczu biznesowym lub często filtrowanych kolumnach
- **Auto Optimize**: Włącz dla Silver/Gold (częste małe zapisy)
- **Gold Incremental**: Używaj MERGE lub partition overwrite zamiast full table overwrite

**Governance:**
- **Unity Catalog**: Bronze/Silver/Gold jako osobne schemas z różnymi permissions
- **Lineage**: Używaj Delta Lake lineage do śledzenia Bronze → Silver → Gold
- **Retention**: Bronze (3-7 lat), Silver (1-2 lata), Gold (6-12 miesięcy)
- **Access Control**: Bronze (data engineers only), Silver (analytics engineers), Gold (BI/analysts read-only)

---

## Gold Layer Best Practices (Szczegółowo)

**1. Incremental Refresh Strategy**

```python
# ✅ DOBRZE: Incremental refresh tylko dla affected dates
affected_dates = get_new_dates_from_silver()
incremental_gold = aggregate_silver(dates=affected_dates)
merge_into_gold(incremental_gold)

# ❌ ŹLE: Full table overwrite codziennie (wolne!)
full_gold = aggregate_all_silver()  # Skanuje całą Silver
overwrite_gold(full_gold)
```

**2. Pre-join Dimensions (Denormalization)**

```python
# ✅ DOBRZE: Join w Gold layer (denormalizacja)
gold_sales = (
    silver_orders
    .join(silver_customers, "customer_id")
    .join(silver_products, "product_id")
    .select(
        "order_id",
        "customer_name",  # From dim_customer
        "product_name",   # From dim_product
        "total_amount"
    )
)
# BI queries: SELECT * FROM gold_sales (szybkie, bez joinów)

# ❌ ŹLE: Pozostaw joins dla BI (wolniejsze dashboardy)
# BI musi joinować Gold fact → dim_customer → dim_product każdorazowo
```

**3. Aggregate Pre-computation**

```python
# ✅ DOBRZE: Pre-compute common aggregations
gold_daily_kpi = aggregate_by_day()
gold_monthly_kpi = aggregate_by_month()
gold_by_region = aggregate_by_region()

# BI queries: SELECT * FROM gold_daily_kpi WHERE date = '2025-01-20' (instant!)

# ❌ ŹLE: Force BI to aggregate on-the-fly
# SELECT SUM(amount) FROM silver_orders WHERE ... (slow dla large data)
```

**4. Gold Monitoring & Alerting**

```python
# ✅ DOBRZE: Continuous monitoring
def gold_quality_checks():
    checks = {
        "freshness": check_data_freshness(gold_table, sla_hours=24),
        "completeness": check_date_continuity(gold_table),
        "reconciliation": check_revenue_match(silver, gold),
        "orphans": check_foreign_keys(gold_fact, dimensions)
    }
    
    if any(not check for check in checks.values()):
        send_alert("Gold Quality Failed", checks)
```

**5. Partitioning Strategy dla Gold**

```python
# ✅ DOBRZE: Partition po BI query patterns
# Jeśli 90% queries: WHERE order_date = '...' AND region = '...'
gold_df.write.partitionBy("order_date", "region").saveAsTable(gold_table)

# ⚠️ Trade-off: Balance partition count vs partition size
# 2 years * 365 days * 5 regions = 3,650 partitions
# Ensure każda partycja > 1 GB
```

---

## Troubleshooting

**Problem 1: Small files w Bronze**
**Objawy:** Setki małych plików Parquet po każdym ingeście

**Rozwiązanie:**
```python
# Włącz Auto Optimize dla Bronze
spark.sql(f"""
    ALTER TABLE {bronze_table} 
    SET TBLPROPERTIES (
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true'
    )
""")
```

---

**Problem 2: Silver processing zbyt wolny**
**Objawy:** Transformacja Bronze → Silver trwa godziny zamiast minut

**Rozwiązanie:** Użyj incremental processing z watermark zamiast full table scan:
```python
# Proces tylko rekordy nowsze niż ostatni Silver timestamp
max_silver_ts = spark.table(silver_table).agg(F.max("_bronze_ingest_timestamp")).collect()[0][0]
new_bronze = spark.table(bronze_table).filter(F.col("_bronze_ingest_timestamp") > max_silver_ts)

# Process only new Bronze records
process_to_silver(new_bronze)
```

---

**Problem 3: Gold re-aggregation trwa zbyt długo**
**Objawy:** Pełna re-agregacja Gold trwa kilka godzin, BI dashboardy czekają

**Rozwiązanie:** Użyj MERGE zamiast overwrite dla incremental Gold:
```python
# Identyfikuj tylko affected dates
affected_dates = new_silver.select("order_date").distinct().collect()

# Re-aggregate tylko affected dates
incremental_gold = aggregate_silver(dates=affected_dates)

# MERGE tylko affected partitions
merge_into_gold(incremental_gold)
```

---

**Problem 4: Gold dashboards pokazują stare dane**
**Objawy:** BI reports są opóźnione o 2+ dni, użytkownicy narzekają

**Diagnoza:**
```python
# Check Gold freshness
max_gold_date = spark.table(gold_table).agg(F.max("order_date")).collect()[0][0]
max_silver_date = spark.table(silver_table).agg(F.max("order_date")).collect()[0][0]

lag_days = (max_silver_date - max_gold_date).days
print(f"Gold is {lag_days} days behind Silver")
```

**Rozwiązanie:**
1. Check Databricks Job logs (czy Gold refresh job failed?)
2. Implement alerting dla Gold freshness SLA
3. Automate Gold refresh (daily cron lub event-driven)

---

**Problem 5: Revenue mismatch między Silver i Gold**
**Objawy:** `SUM(silver.total_amount) ≠ SUM(gold.total_revenue)`

**Diagnoza:**
```python
# Reconciliation check
silver_revenue = spark.table(silver_table).agg(F.sum("total_amount")).collect()[0][0]
gold_revenue = spark.table(gold_table).agg(F.sum("total_revenue")).collect()[0][0]
variance = abs(silver_revenue - gold_revenue)

print(f"Variance: ${variance:,.2f}")

# Find problematic dates
by_date_check = (
    spark.table(silver_table)
    .groupBy("order_date")
    .agg(F.sum("total_amount").alias("silver_revenue"))
    .join(
        spark.table(gold_table).groupBy("order_date").agg(F.sum("total_revenue").alias("gold_revenue")),
        "order_date",
        "full_outer"
    )
    .withColumn("variance", F.col("silver_revenue") - F.col("gold_revenue"))
    .filter(F.abs(F.col("variance")) > 0.01)
)

display(by_date_check)
```

**Możliwe przyczyny:**
- Gold aggregation bug (incorrect groupBy key)
- Missing partitions w Gold (niektóre daty nie zagregowane)
- Double counting (duplikaty w Gold)
- Filtry w Gold (np. `WHERE status = 'COMPLETED'` ale Silver ma wszystkie statusy)

---

**Problem 6: Orphan records w Gold (unmatched foreign keys)**
**Objawy:** Gold fact table ma `customer_id` którego nie ma w `dim_customer`

**Diagnoza:**
```python
# Find orphan customers
orphan_customers = (
    gold_fact
    .select("customer_id")
    .distinct()
    .join(dim_customer.select("customer_id"), "customer_id", "left_anti")
)

orphan_count = orphan_customers.count()
print(f"Orphan customers: {orphan_count}")

if orphan_count > 0:
    display(orphan_customers.limit(10))
```

**Rozwiązanie:**
1. **Left join z NULL handling**:
```python
gold_with_safeguard = (
    silver_orders
    .join(dim_customer, "customer_id", "left")  # Left join zamiast inner
    .withColumn("customer_name", F.coalesce(F.col("customer_name"), F.lit("UNKNOWN")))
)
```

2. **Data quality check w Silver → Gold**:
```python
# Reject orders z invalid customer_id
valid_customer_ids = dim_customer.select("customer_id").distinct()
validated_orders = silver_orders.join(valid_customer_ids, "customer_id", "inner")
```

3. **Alert dla nowych orphans**:
```python
if orphan_count > threshold:
    send_alert(f"Orphan customers detected: {orphan_count}")
```

---

## Podsumowanie

**W tym notebooku nauczyliśmy się:**

✅ **Medallion Architecture:**
- Bronze: Raw data landing zone z audit metadata (immutable, append-only)
- Silver: Cleansed, validated, deduplicated data (business rules enforcement)
- Gold: Business-level aggregates i KPI (BI-ready, denormalized)

✅ **ETL vs ELT:**
- ELT approach: Load first (Bronze), then Transform (Silver/Gold)
- Możliwość reprocessingu bez re-ingestion
- Incremental processing dla każdej warstwy

✅ **Partitioning Strategy:**
- Partycjonuj tylko duże tabele (>1TB)
- Partition size > 1GB dla uniknięcia small files
- Silver/Gold: partycjonowanie po dacie lub business dimensions

**Kluczowe wnioski:**
1. Medallion Architecture zapewnia separation of concerns i data quality gates
2. Każda warstwa ma określone SLA, retention policy i access patterns
3. Bronze jako immutable source of truth umożliwia reprocessing
4. Incremental processing jest kluczowy dla performance w dużej skali

**Następne kroki:**
- **Kolejny notebook**: 03_batch_streaming_load.ipynb - COPY INTO, Auto Loader, Structured Streaming
- **Warsztat praktyczny**: 01_delta_medallion_workshop.ipynb
- **Delta Live Tables**: Automatyczna implementacja Medallion z deklaratywnym API

---

## Cleanup

Opcjonalnie: usuń utworzone tabele Demo po zakończeniu ćwiczeń:

In [0]:
# Opcjonalne czyszczenie zasobów testowych
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# spark.sql(f"DROP TABLE IF EXISTS {bronze_table}")
# spark.sql(f"DROP TABLE IF EXISTS {silver_table}")
# spark.sql(f"DROP TABLE IF EXISTS {silver_partitioned_table}")
# spark.sql(f"DROP TABLE IF EXISTS {gold_table}")

# spark.catalog.clearCache()
# print("Zasoby zostały wyczyszczone")